In [1]:
import pandas as pd
import re
import spacy
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download("punkt_tab")

!py -m spacy download es_core_news_sm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aml\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aml\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aml\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------  12.8/12.9 MB 67.6 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 36.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\aml\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [3]:
nlp = spacy.load("es_core_news_sm")
stop_words = set(nltk.corpus.stopwords.words('spanish'))

In [4]:
def process_and_normalize(documents):
    """
    Combina tokenización y normalización en un solo paso.

    Args:
        documents (list): Una lista de cadenas de texto.

    Returns:
        list: Una lista de documentos normalizados listos para vectorizar.
    """
    normalized_list = []
    for doc in documents:
        # Pasa el documento completo por el pipeline de spaCy
        # Esto tokeniza, lematiza, y etiqueta el texto
        processed_doc = nlp(re.sub(r'[^\w\s]', '', doc.lower()))

        # Filtra los tokens para eliminar stopwords y no alfabéticos
        normalized_words = [
            token.lemma_ for token in processed_doc
            if token.is_alpha and token.text not in stop_words
        ]
        # Une las palabras normalizadas en una sola cadena
        normalized_list.append(" ".join(normalized_words))
    return normalized_list

In [5]:
corpus = [
    "El gato negro salta sobre la mesa.",
    "El perro salta sobre la cerca."
]

print("1. Corpus original:")
print(corpus)

# Tokenización y Normalización ---
normalized_corpus = process_and_normalize(corpus)

print("\n2. Corpus normalizado (lemas):")
print(normalized_corpus)

# Representación Numérica ---
## Representación con Bag of Words (BoW)
print("\n--- Bag of Words (BoW) ---")
vectorizer_bow = CountVectorizer()
bow_matrix = vectorizer_bow.fit_transform(normalized_corpus)

df_bow = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer_bow.get_feature_names_out())
df_bow.index = [f"Doc {i+1}" for i in range(len(corpus))]

print("Matriz de conteos (frecuencia de palabras):")
print(df_bow)

## Representación con TF-IDF
print("\n--- TF-IDF ---")
vectorizer_tfidf = TfidfVectorizer()
tfidf_matrix = vectorizer_tfidf.fit_transform(normalized_corpus)

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer_tfidf.get_feature_names_out())
df_tfidf.index = [f"Doc {i+1}" for i in range(len(corpus))]

print("Matriz de valores TF-IDF:")
print(df_tfidf)

# Análisis y Comparación ---
print("\n4. Comparación y conclusiones:")
print("Las matrices de BoW y TF-IDF tienen el mismo vocabulario, pero sus valores son diferentes.")
print("- BoW: Usa conteos simples. Las palabras como 'saltar' tienen el mismo peso en ambos documentos (1).")
print("- TF-IDF: Asigna un peso mayor a las palabras distintivas. 'gato' y 'negro' en el Doc 1 y 'perro' y 'cerca' en el Doc 2 tienen valores más altos, haciéndolas más importantes para la distinción de los documentos.")

1. Corpus original:
['El gato negro salta sobre la mesa.', 'El perro salta sobre la cerca.']

2. Corpus normalizado (lemas):
['gato negro salta mesa', 'perro salta cerca']

--- Bag of Words (BoW) ---
Matriz de conteos (frecuencia de palabras):
       cerca  gato  mesa  negro  perro  salta
Doc 1      0     1     1      1      0      1
Doc 2      1     0     0      0      1      1

--- TF-IDF ---
Matriz de valores TF-IDF:
          cerca      gato      mesa     negro     perro     salta
Doc 1  0.000000  0.534046  0.534046  0.534046  0.000000  0.379978
Doc 2  0.631667  0.000000  0.000000  0.000000  0.631667  0.449436

4. Comparación y conclusiones:
Las matrices de BoW y TF-IDF tienen el mismo vocabulario, pero sus valores son diferentes.
- BoW: Usa conteos simples. Las palabras como 'saltar' tienen el mismo peso en ambos documentos (1).
- TF-IDF: Asigna un peso mayor a las palabras distintivas. 'gato' y 'negro' en el Doc 1 y 'perro' y 'cerca' en el Doc 2 tienen valores más altos, haciéndo